In [1]:
from joblib import Parallel, delayed
from math import sqrt
from pathlib import Path
from lxml import etree
from tqdm import tqdm
import pickle
import dateparser

from bs4 import BeautifulSoup

import pandas as pd
import swifter

import pandas as pd
import sqlite3
from cleantext import clean

from pathlib import Path
import numpy as np
import swifter
from somajo import Tokenizer, SentenceSplitter
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()

import numpy as np

/home/filter/anaconda3/envs/ptf/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
df1 = pd.read_pickle('comments.pkl')

In [3]:
df2 = pd.read_pickle('comments_more.pkl')

In [4]:
df2.columns = ['text', 'date']

In [5]:
df = pd.concat([df1, df2])

In [7]:
def parse(x):
    idx = x.find('—')
    return dateparser.parse(x[idx:], languages=['de'])

In [8]:
parsed = Parallel(n_jobs=4)(delayed(parse)(i) for i in tqdm(df['date'].values))

  0%|          | 10289/6405790 [00:06<1:06:27, 1603.74it/s]

KeyboardInterrupt: 

In [41]:
df['parsed'] = parsed

In [43]:
df.to_pickle('fin_com.pkl')

In [9]:
df = pd.read_pickle('fin_com.pkl')

  0%|          | 10289/6405790 [00:20<1:06:27, 1603.74it/s]

In [38]:
def get_sents(texts):
    tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
    sentence_splitter = SentenceSplitter(is_tuple=False)
    
    results = []
    for text in texts:
        text = clean(text, lang='de', lower=False)
        tokens = tokenizer.tokenize_paragraph(text)
        sentences = sentence_splitter.split(tokens)
        cleaned = [clean(' '.join(s), no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de') for s in sentences]
        results.append(cleaned)
    return results

In [39]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [40]:
df

,text,date,parsed
0,"Wer darf Atommacht sein, wer nicht. Da treffe...","#25 — 3. April 2015, 16:21 Uhr",2015-04-03 16:21:00.000000
0,"(i.) Ist ""Satoshi Nakamoto"" nicht der Name ein...","#65 — 9. Dezember 2017, 9:17 Uhr",2017-12-09 09:17:00.000000
1,"P.S.:\nWer wissen möchte, wie die virtuell ""be...","#65.1 — 9. Dezember 2017, 9:32 Uhr",2017-12-09 09:32:00.000000
2,Die Idee von bitcoin ist doch die Abschaffung ...,"#66 — 9. Dezember 2017, 9:21 Uhr",2017-12-09 09:21:00.000000
3,Bitcoins wird auf Null oder nahe Null fallen.D...,"#66.1 — 9. Dezember 2017, 9:31 Uhr",2017-12-09 09:31:00.000000
4,Schon zweihundert Kommentare. Das zeigt tatsäc...,"#67 — 9. Dezember 2017, 11:25 Uhr",2017-12-09 11:25:00.000000
5,"Wer damit reales Geld macht, hat es verdient. ...","#68 — 9. Dezember 2017, 18:44 Uhr",2017-12-09 18:44:00.000000
0,"""Derzeit vergeht kein Tag ohne Facebook-Schlag...",#17 — vor 9 Monaten,2018-08-22 22:52:00.811079
1,"Wer entscheidet, was Fake News sind? Herr Kle...",#18 — vor 9 Monaten,2018-08-22 22:52:00.812714
2,Für anonyme Accounts gibt es keine Meinungsfre...,#18.1 — vor 9 Monaten,2018-08-22 22:52:00.813975


In [41]:
df['text'].values[:1]

array(['Wer darf Atommacht sein, wer nicht.  Da treffen sich Regierungen, Atommächte und Anwärter auf die Bombe.\nÜber was wird da verhandelt? \nDie Bombe selbst steht nicht zur Disposition, für die Atommächte, sondern der Verfügungsanspruch eines Anwärters.\nDer Besitz dieser Bombe verändert die politischen Optionen eines Staates, und damit auch die militärischen , in der Folge die politischen Optionen der existierenden Atommächte.\nBereits der Wille zur Bombe wird deshalb von den realen Atommächten beaufsichtigt. Diese Mächte verhalten sich zum Willen eines ausländischen Souveräns wie Polizei. Wer nicht gehorcht wird bestraft. \nDas können diese Mächte, weil diese in der Lage sind ihre Ansprüche an das Wohlverhalten anderer Regierungen wirtschaftlich und militärisch zu erzwingen.\nVon wegen hier ginge es um den Schutz vor einer militärischen Bedrohung.\nDie Fähigkeit zu atomarer Eskalation steht doch nur für den Angeklagten zur Disposition.\nWas bleibt? Die auch atomare Überlegenheit

In [42]:
df['text'] = df['text'].astype(str)

In [43]:
df['text'].values

array(['Wer darf Atommacht sein, wer nicht.  Da treffen sich Regierungen, Atommächte und Anwärter auf die Bombe.\nÜber was wird da verhandelt? \nDie Bombe selbst steht nicht zur Disposition, für die Atommächte, sondern der Verfügungsanspruch eines Anwärters.\nDer Besitz dieser Bombe verändert die politischen Optionen eines Staates, und damit auch die militärischen , in der Folge die politischen Optionen der existierenden Atommächte.\nBereits der Wille zur Bombe wird deshalb von den realen Atommächten beaufsichtigt. Diese Mächte verhalten sich zum Willen eines ausländischen Souveräns wie Polizei. Wer nicht gehorcht wird bestraft. \nDas können diese Mächte, weil diese in der Lage sind ihre Ansprüche an das Wohlverhalten anderer Regierungen wirtschaftlich und militärisch zu erzwingen.\nVon wegen hier ginge es um den Schutz vor einer militärischen Bedrohung.\nDie Fähigkeit zu atomarer Eskalation steht doch nur für den Angeklagten zur Disposition.\nWas bleibt? Die auch atomare Überlegenheit

In [ ]:
results = Parallel(n_jobs=4)(delayed(get_sents)(row) for row in tqdm(list(chunks(df['text'].values, 1000))))








  0%|          | 0/6406 [00:00<?, ?it/s]






  0%|          | 1/6406 [00:00<37:45,  2.83it/s]






  0%|          | 8/6406 [00:07<57:01,  1.87it/s]






  0%|          | 9/6406 [00:08<1:14:41,  1.43it/s]






  0%|          | 10/6406 [00:08<56:37,  1.88it/s] 






  0%|          | 12/6406 [00:13<1:58:01,  1.11s/it]






  0%|          | 13/6406 [00:14<2:06:32,  1.19s/it]






  0%|          | 14/6406 [00:15<1:53:24,  1.06s/it]






  0%|          | 15/6406 [00:15<1:28:42,  1.20it/s]






  0%|          | 16/6406 [00:19<3:08:48,  1.77s/it]






  0%|          | 18/6406 [00:21<2:43:44,  1.54s/it]






  0%|          | 19/6406 [00:22<2:32:42,  1.43s/it]






  0%|          | 20/6406 [00:25<3:23:38,  1.91s/it]






  0%|          | 21/6406 [00:26<2:47:18,  1.57s/it]






  0%|          | 22/6406 [00:27<2:23:18,  1.35s/it]






  0%|          | 23/6406 [00:29<2:34:23,  1.45s/it]






  0%|          | 24/6406 [00:32<3:28:04,  1.96s/it]






  0%|          | 25/6406 [

In [23]:
results

[['wer darf atommacht sein wer nicht',
  'da treffen sich regierungen atommächte und anwärter auf die bombe',
  'über was wird da verhandelt',
  'die bombe selbst steht nicht zur disposition für die atommächte sondern der verfügungsanspruch eines anwärters',
  'der besitz dieser bombe verändert die politischen optionen eines staates und damit auch die militärischen in der folge die politischen optionen der existierenden atommächte',
  'bereits der wille zur bombe wird deshalb von den realen atommächten beaufsichtigt',
  'diese mächte verhalten sich zum willen eines ausländischen souveräns wie polizei',
  'wer nicht gehorcht wird bestraft',
  'das können diese mächte weil diese in der lage sind ihre ansprüche an das wohlverhalten anderer regierungen wirtschaftlich und militärisch zu erzwingen',
  'von wegen hier ginge es um den schutz vor einer militärischen bedrohung',
  'die fähigkeit zu atomarer eskalation steht doch nur für den angeklagten zur disposition',
  'was bleibt',
  'die au

In [2]:
cleaned = pd.read_pickle('/mnt/data2/ptf/cleaned.pkl')

In [3]:
cleaned = cleaned[['cleaned']]

In [4]:
cleaned = cleaned['cleaned'].values

In [14]:
cnt = 0
for d in cleaned:
    cnt += len([x for x in d if len(x) > 0])

In [16]:
len(cleaned)

6405790

In [15]:
cnt

27152024

In [7]:
with open('zo.txt', 'w') as outfile:
    for d in cleaned:
        outfile.write('\n'.join(d))

In [10]:
cleaned.cleaned.str.len().min()

1

In [15]:
cleaned.values[0]

array([list(['wer darf atommacht sein wer nicht', 'da treffen sich regierungen atommächte und anwärter auf die bombe', 'über was wird da verhandelt', 'die bombe selbst steht nicht zur disposition für die atommächte sondern der verfügungsanspruch eines anwärters', 'der besitz dieser bombe verändert die politischen optionen eines staates und damit auch die militärischen in der folge die politischen optionen der existierenden atommächte', 'bereits der wille zur bombe wird deshalb von den realen atommächten beaufsichtigt', 'diese mächte verhalten sich zum willen eines ausländischen souveräns wie polizei', 'wer nicht gehorcht wird bestraft', 'das können diese mächte weil diese in der lage sind ihre ansprüche an das wohlverhalten anderer regierungen wirtschaftlich und militärisch zu erzwingen', 'von wegen hier ginge es um den schutz vor einer militärischen bedrohung', 'die fähigkeit zu atomarer eskalation steht doch nur für den angeklagten zur disposition', 'was bleibt', 'die auch atomare üb